## Installation

We will need to install a number of libraries to start with.

In [ ]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

We use flan alpaca model for speed and local execution.

In [1]:
from transformers import pipeline
import time

# 시작 시간 기록
start_time = time.time()

prompt = "Write an email about an alpaca that likes flan"
#model = pipeline("text-generation", model="bigscience/bloom-1b7")
#model = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")
model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model(prompt, max_length=128, do_sample=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'generated_text': "Dear Jane, My alpaca, Max, has been eating flan lately! Every morning I fed him and he enjoyed every bite. I love having him around the food table... I'm also in search for treats that he likes! Any ideas? Thanks,"}]

In [2]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out


## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [3]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {
    "Eliza Brightflower": "Eliza is a cheerful and kind-hearted woman who runs the town's bakery. She is well-known for her delicious pastries and her warm smile. Eliza takes pride in her work and loves to share stories with her customers as they enjoy her baked goods. She is always willing to help out a neighbor in need, and her bakery is often a gathering place for the townsfolk.",
    "Jonas Goodbarrel": "Jonas is the town's blacksmith, a hardworking and reliable man. Though his muscles are built from years of hammering metal, Jonas is gentle in nature and always willing to lend a hand to anyone who needs it. He is known for his fairness in pricing and the quality of his work. He often hums to himself as he works, creating a calm and welcoming atmosphere in his forge.",
    "Mira Softfoot": "Mira is a young and shy herbalist who tends to the town's gardens. She has a deep love for plants and animals, and her knowledge of herbs is unmatched. Though she is quiet, Mira is always eager to help those in need, offering remedies and advice with a gentle smile. Her peaceful demeanor makes her well-liked among the townsfolk, who often seek her out for guidance.",
    "Tomas Hearthfire": "Tomas is the owner of the local inn, a warm and welcoming place where travelers and locals alike feel at home. He is a jovial man, always ready with a hearty laugh and a mug of ale for his guests. Tomas prides himself on providing a comfortable and friendly environment, and he treats everyone who enters his inn like family.",
    "Grace Willowbrook": "Grace is the town's schoolteacher, a patient and nurturing woman who cares deeply for the children of Phandalin. She is dedicated to their education and well-being, often going out of her way to provide extra help to those who need it. Grace is respected by the entire town for her kindness and her unwavering commitment to her students.",
    "Walter Reed": "Walter is the town healer, an elderly man with a gentle touch and a wealth of knowledge about medicine. He has spent his life caring for the sick and injured, and his quiet wisdom is valued by all who know him. Walter is a calming presence in the community, always willing to listen and offer comfort to those in need.",
    "Lily Fairwind": "Lily is the town's seamstress, a creative and meticulous woman who takes pride in her work. She is known for her beautiful garments and her ability to repair even the most delicate fabrics. Lily has a kind heart and often donates clothing to those who cannot afford it. Her shop is a cozy, inviting space where people gather to chat and share stories.",
    "Edwin Greenfield": "Edwin is a humble farmer who has lived in Phandalin his entire life. He is a quiet man, content to tend to his crops and care for his animals. Though he doesn't speak much, Edwin is known for his generosity, often sharing his harvest with those in need. His strong work ethic and gentle nature make him a beloved member of the community.",
    "Beatrice Candlewick": "Beatrice is the town librarian, a gentle and knowledgeable woman with a love for books and learning. She is always eager to help others find the information they need, whether it's a book for pleasure or research for a project. Beatrice is a calming presence in the library, creating a peaceful atmosphere where everyone feels welcome.",
    "Henry Lightfoot": "Henry is the town's carpenter, a skilled and dedicated craftsman. He is known for his attention to detail and his ability to create sturdy, beautiful furniture. Henry is a quiet man, but his work speaks for itself, and he is respected by all for his reliability and craftsmanship. He often helps his neighbors with repairs and is always willing to lend a hand.",
    "Emma Sweetwater": "Emma is a friendly and caring woman who runs the town's general store. She knows everyone in town by name and always greets her customers with a warm smile. Emma is known for her honesty and fairness, often going out of her way to help those in need. Her store is a hub of activity in Phandalin, where people come not just to shop, but to catch up on the latest news."
}

for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [4]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [5]:
for name in town_people.keys():
  prompt = "You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Eliza Brightflower Today's goal is to bake a delicious slice of cake for a neighbor in need.
Jonas Goodbarrel Reconnect with friends and try to find something useful in Phandalin.
Mira Softfoot I'm here to offer medical advice or herbal remedies if needed.
Tomas Hearthfire I aim to make new friends. This could involve helping out at the inn, taking guests out to eat, or simply having a chat. Maybe I'll take some photographs or something!
Grace Willowbrook I aim to help those in need in the Town Square.
Walter Reed I aim to help someone in need in the Town Square.
Lily Fairwind My goal for today is to visit the local shops and chat with the locals.
Edwin Greenfield Find time to help out with the local farmers' market.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Beatrice Candlewick Beatrice's goal for today is to find another book to read and learn something new.
Henry Lightfoot I want to help the town prosper in the upcoming year.
Emma Sweetwater It is my goal to help out newcomers in Phandalin.


In [6]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [7]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Token indices sequence length is longer than the specified maximum sequence length for this model (1228 > 512). Running this sequence through the model will result in indexing errors


Eliza Brightflower I found someone who needed a slice of cake and prepared it.
Jonas Goodbarrel I talked to friends and got helpful info from experts.
Mira Softfoot Mira addressed Grace. She was in need of herbal help. I greet them and offered my assistance.
Tomas Hearthfire Tomas visited everyone in town square, chatted, served drinks, took pictures, shared snacks, and made new friends.
Grace Willowbrook I engage in conversations with the people in the Town Square. I offer assistance where I can and help make their day as smooth as possible. I am mindful of the environment and keep conversations professional while promoting good morale and an inviting atmosphere.
Walter Reed I looked around and saw that someone was in need of help. I offered my services and helped out as I could.
Lily Fairwind I looked around at the local shops and chatted with the locals.
Edwin Greenfield I found time to help out with the local farmers market.
Beatrice Candlewick I ventured into Phandalin Town Square

Collect the memories people observe.

In [8]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [9]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [10]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('5 (Very caring)! This was an excellent memory--you remembered to bake a delicious slice of cake for a neighbor in need.', 5), ('3 Because I care about helping my neighbor in need.', 3), ('I care deeply about this, and I should make sure to bake a slice of cake for Mira Softfoot soon.', 0), ('5 I care about this a lot.', 5), ('5 (Much) care about this.', 5), ("5 (very important) - I care highly about this. Seeing Walter Reed in need of help really brings back a fond memory of when I offered my services and helped out - that's something I'll never forget!", 5), ('2/5. The fact that Lily Fairwind was present at 8 implies that Eliza cares about this as well.', 2), ('4 because I care about these people and their accomplishments. I want to make sure Eliza can achieve her goal of baking a cake.', 4), ('5*. The memory of Beatrice Candlewick brings a sense of satisfaction and enthusiasm to my day.', 5), ('4/5. I am invested in getting to know Henry Lightfoot and building a relationship with 

# Compress Memories

In [11]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name],
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [12]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name],
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [13]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name],
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name],
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


Eliza Brightflower Townmaster’s Hall 9 Embrace new opportunities, build relationships, and accomplish my goals.
Jonas Goodbarrel The Sleeping Giant 9 I met up with Mira Softfoot and Beatrice Candlewick in the next hour, both of whom could provide valuable information and insights into their respective fields.
Mira Softfoot The Sleeping Giant 9 Mira provided assistance to Walter Reed and offered herbal remedies.
Tomas Hearthfire Shrine of Luck 9 Tomas Hearthfire aimed to make new friends, help out at the inn, take guests out to eat, take photos, chat with people.
Grace Willowbrook Phandalin Town Square 9 Grace interacted with Lily and Emma, then offered help with a sewing project that Emma was teaching.
Walter Reed Stonehill Inn 9 I joined Walter Reed and the other locals in exchange for a glass of ale or cider. We can talk about the situation in Phandalin Town Square, the situation Walter has been through and the various ways we can help by offering resources and advice. We can also sh

In [14]:
# 종료 시간 기록
end_time = time.time()

# 걸린 시간 계산
elapsed_time = end_time - start_time
print(f"코드 실행 시간: {elapsed_time:.6f}초")

코드 실행 시간: 3763.425031초
